<a href="https://colab.research.google.com/github/Tempate/Prophet/blob/main/mbert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Building a classifier with mBERT

We start by installing the huggingface transformer package.

In [1]:
!pip install transformers

## Preparing the data

We load the dataset.

In [2]:
from google.colab import drive
drive.mount('/content/drive')

with open('/content/drive/My Drive/dataset (1).csv') as file:
  dataset = file.read().splitlines()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


We remove characters that aren't letters.

In [3]:
import regex as re
import random


SAMPLE_COUNT = 19

data = []

for entry in random.sample(dataset, SAMPLE_COUNT):
    text = re.findall('\p{L}+', entry.lower())
    data.append((" ".join(text), int(entry[-1])))

# Remove the dataset from memory
del dataset

## Setting up the model

The idea is to use mBERT as the input vector for the classifier.

In [4]:
import torch.nn as nn


class Classifier(nn.Module):
    def __init__(self, input_size):
        super(Classifier, self).__init__()

        self.function1 = nn.Linear(input_size, 16)
        self.activation1 = nn.ReLU()

        self.function2 = nn.Linear(16, 16)
        self.activation2 = nn.ReLU()

        self.function3 = nn.Linear(16, 1)
        self.activation3 = nn.Sigmoid()

    def forward(self, vector):
        layer1 = self.activation1(self.function1(vector))
        layer2 = self.activation2(self.function2(layer1))
        layer3 = self.activation3(self.function3(layer2))

        return (layer3[0]).float()


We are algo going to create a class for our dataset.

In [5]:
from torch.utils.data import Dataset

import torch


class ReviewDataset(Dataset):
  def __init__(self, review, target, tokenizer, max_length):
    self.review = review
    self.target = target
    self.tokenizer = tokenizer
    self.max_length = max_length

  def __len__(self):
    return len(self.review)

  def __getitem__(self, item):
    review = str(self.review[item])

    encoding = self.tokenizer.encode_plus(
      review,
      
      add_special_tokens=True,
      
      truncation=True,
      padding='max_length',
      max_length=self.max_length,
      
      return_token_type_ids=False,
      return_attention_mask=True,
      return_tensors='pt'
    )

    return {
      'text': review,
      'targets': torch.tensor(self.target[item], dtype=torch.long),

      'attention_mask': encoding['attention_mask'].flatten(),
      'input_ids': encoding['input_ids'].flatten()
    }


## Merging the classifier with mBERT.

This class has two purposes: to vectorize data by passing it to mBERT and to prepare data using DataLoader and our ReviewDataset.

In [6]:
from torch.utils.data import DataLoader

from transformers import BertTokenizer
from transformers import BertModel

import numpy as np
import torch


NUM_WORKERS = 2
MAX_LENGTH = 128
BATCH_SIZE = 1


class Vectorizer():
    def __init__(self, MODEL):
        self.tokenizer = BertTokenizer.from_pretrained(MODEL)
        self.model = BertModel.from_pretrained(MODEL)

    def vectorize(self, data):
        self.vectors = {}

        for entry in data:
            for batch in self.data_loader([entry]):
                
                with torch.no_grad():
                    id = batch["text"][0]
                    
                    self.vectors[id] = self.model(
                        input_ids=batch["input_ids"], 
                        attention_mask=batch["attention_mask"]
                    )[1]

        print("[+] Data vectorized correctly")

    def data_loader(self, data):
        texts, labels = zip(*data)

        entry = ReviewDataset(
            np.array(texts),
            np.array(labels),
            self.tokenizer,
            MAX_LENGTH
        )

        loader = DataLoader(
            entry, 
            batch_size=BATCH_SIZE, 
            num_workers=NUM_WORKERS
        )

        return loader


## Training the model

This class combines all the previous classes to train and validate our model.

In [7]:
from torch.utils.data import DataLoader
from sklearn.metrics import f1_score

import torch
import torch.nn as nn
import torch.optim as optim

import numpy as np


MODEL = 'bert-base-multilingual-uncased'

LEARNING_RATE = 5e-3


class Transformers:

    def __init__(self, data):
        self.vectorizer = Vectorizer(MODEL)
        self.vectorizer.vectorize(data)

        self.model = Classifier(self.vectorizer.model.config.hidden_size)

    def train(self, train_data, valid_data, epochs, patience):
        optimizer = optim.NAdam(self.model.parameters(), lr=LEARNING_RATE)
        loss_function = nn.BCELoss()

        losses = []

        # We pass several times over the training data.
        # Usually there are between 5 and 30 epochs.
        for epoch in range(epochs):
            for review in train_data:
              for batch in self.vectorizer.data_loader([review]):
                  
                  vector = self.vectorizer.vectors[batch["text"][0]]
                  target = batch["targets"].float()

                  # We clear the gradients before each instance
                  self.model.zero_grad()

                  # We run the forward pass
                  log_probs = self.model(vector)

                  # We compute the loss, gradients, and update the parameters
                  loss = loss_function(log_probs, target)
                  loss.backward()

                  optimizer.step()

            score, loss = self.validate(valid_data, loss_function)
            losses.append(loss)

            print(f"{epoch + 1}.\tLoss: {loss}\tF1-Score: {score}")

            # We count the patience to avoid over-fitting
            if len(losses) >= 2 and losses[-1] - losses[-2] >= 0:
                patience -= 1

            if patience <= 0:
                break

        return score, loss

    def validate(self, data, loss_function):
        targets = []
        guesses = []

        loss = 0

        batches = self.vectorizer.data_loader(data)
        
        for batch in batches:

            vector = self.vectorizer.vectors[batch["text"][0]]
            
            target = batch["targets"].float()
            targets.append(target)

            with torch.no_grad():
                # We run the forward pass
                output = self.model(vector)

                # We save our prediction
                guess = torch.round(torch.flatten(output))
                guesses.append(guess)
                
                # We calculate the loss
                loss += loss_function(output, target).item()

        score = f1_score(targets, guesses, zero_division=1)
        loss /= len(batches)

        return score, loss


## Testing our model

This approach is good when testing with a tiny dataset. In our case we only have 19 entries.

In [ ]:
NUMBER_OF_TIMES_TO_TEST = 5
EPOCHS = 100
PATIENCE = 10

def leave_one_out(data):
    total_score = 0

    for n in range(NUMBER_OF_TIMES_TO_TEST):
        score = 0

        for i in range(len(data)):
            train = data[:i] + data[i+1:]
            valid = [data[i]]

            model = Transformers(data)
            score += model.train(train, valid, EPOCHS, PATIENCE)[0]

        score /= len(data)
        total_score += score

        print("[%d]\t%.4f" % (n + 1, score))

    print("Average score: %.4f" % (total_score / NUMBER_OF_TIMES_TO_TEST))


leave_one_out(data)

Some weights of the model checkpoint at bert-base-multilingual-uncased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


[+] Data vectorized correctly
1.	Loss: 0.6190440058708191	F1-Score: 1.0
2.	Loss: 0.6032919883728027	F1-Score: 1.0
3.	Loss: 0.5857675671577454	F1-Score: 1.0
4.	Loss: 0.5696262717247009	F1-Score: 1.0
5.	Loss: 0.5541063547134399	F1-Score: 1.0
6.	Loss: 0.542468786239624	F1-Score: 1.0
7.	Loss: 0.5332370400428772	F1-Score: 1.0
8.	Loss: 0.5258433222770691	F1-Score: 1.0
9.	Loss: 0.5199636816978455	F1-Score: 1.0
10.	Loss: 0.5150814652442932	F1-Score: 1.0
11.	Loss: 0.5110149383544922	F1-Score: 1.0
12.	Loss: 0.5076237916946411	F1-Score: 1.0
13.	Loss: 0.5047928690910339	F1-Score: 1.0
14.	Loss: 0.5024266839027405	F1-Score: 1.0
15.	Loss: 0.5004462599754333	F1-Score: 1.0
16.	Loss: 0.4987863004207611	F1-Score: 1.0
17.	Loss: 0.4973927140235901	F1-Score: 1.0
18.	Loss: 0.4962209463119507	F1-Score: 1.0
19.	Loss: 0.4952346682548523	F1-Score: 1.0
20.	Loss: 0.494403600692749	F1-Score: 1.0
21.	Loss: 0.49370276927948	F1-Score: 1.0
22.	Loss: 0.49311643838882446	F1-Score: 1.0
23.	Loss: 0.4926508963108063	F1-Scor

Some weights of the model checkpoint at bert-base-multilingual-uncased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


[+] Data vectorized correctly
1.	Loss: 0.6212506294250488	F1-Score: 1.0
2.	Loss: 0.5083303451538086	F1-Score: 1.0
3.	Loss: 0.48239952325820923	F1-Score: 1.0
4.	Loss: 0.48061835765838623	F1-Score: 1.0
5.	Loss: 0.481477826833725	F1-Score: 1.0
6.	Loss: 0.48208749294281006	F1-Score: 1.0
7.	Loss: 0.4824933707714081	F1-Score: 1.0
8.	Loss: 0.4827989637851715	F1-Score: 1.0
9.	Loss: 0.48301786184310913	F1-Score: 1.0
10.	Loss: 0.4831470549106598	F1-Score: 1.0
11.	Loss: 0.48273220658302307	F1-Score: 1.0
12.	Loss: 0.48249125480651855	F1-Score: 1.0
13.	Loss: 0.4823012351989746	F1-Score: 1.0
14.	Loss: 0.4820607602596283	F1-Score: 1.0
15.	Loss: 0.48172634840011597	F1-Score: 1.0
16.	Loss: 0.4817485511302948	F1-Score: 1.0
17.	Loss: 0.4814371168613434	F1-Score: 1.0
18.	Loss: 0.4802631735801697	F1-Score: 1.0
19.	Loss: 0.4793137013912201	F1-Score: 1.0
20.	Loss: 0.47931215167045593	F1-Score: 1.0
21.	Loss: 0.4780392646789551	F1-Score: 1.0
22.	Loss: 0.4768325090408325	F1-Score: 1.0
23.	Loss: 0.47566232085227

Some weights of the model checkpoint at bert-base-multilingual-uncased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


...

[+] Data vectorized correctly
1.	Loss: 0.8819653391838074	F1-Score: 0.0
2.	Loss: 1.0746209621429443	F1-Score: 0.0
3.	Loss: 1.101270318031311	F1-Score: 0.0
4.	Loss: 1.099574089050293	F1-Score: 0.0
5.	Loss: 1.1000518798828125	F1-Score: 0.0
6.	Loss: 1.1026333570480347	F1-Score: 0.0
7.	Loss: 1.1006648540496826	F1-Score: 0.0
8.	Loss: 1.0985887050628662	F1-Score: 0.0
9.	Loss: 1.0971416234970093	F1-Score: 0.0
10.	Loss: 1.0961401462554932	F1-Score: 0.0
11.	Loss: 1.095442771911621	F1-Score: 0.0
12.	Loss: 1.0949382781982422	F1-Score: 0.0
13.	Loss: 1.094617486000061	F1-Score: 0.0
14.	Loss: 1.0943593978881836	F1-Score: 0.0
15.	Loss: 1.0948278903961182	F1-Score: 0.0
16.	Loss: 1.0945258140563965	F1-Score: 0.0
17.	Loss: 1.0944856405258179	F1-Score: 0.0
18.	Loss: 1.0945732593536377	F1-Score: 0.0
19.	Loss: 1.0947494506835938	F1-Score: 0.0
20.	Loss: 1.0949630737304688	F1-Score: 0.0
21.	Loss: 1.0952180624008179	F1-Score: 0.0
22.	Loss: 1.095518708229065	F1-Score: 0.0


Some weights of the model checkpoint at bert-base-multilingual-uncased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


[+] Data vectorized correctly
1.	Loss: 0.4395131468772888	F1-Score: 1.0
2.	Loss: 0.5323485136032104	F1-Score: 1.0
3.	Loss: 0.5437688231468201	F1-Score: 1.0
4.	Loss: 0.5382479429244995	F1-Score: 1.0
5.	Loss: 0.5312632322311401	F1-Score: 1.0
6.	Loss: 0.5287757515907288	F1-Score: 1.0
7.	Loss: 0.5272478461265564	F1-Score: 1.0
8.	Loss: 0.526122510433197	F1-Score: 1.0
9.	Loss: 0.5248066782951355	F1-Score: 1.0
10.	Loss: 0.5237318277359009	F1-Score: 1.0
11.	Loss: 0.5224337577819824	F1-Score: 1.0
12.	Loss: 0.5205456018447876	F1-Score: 1.0
13.	Loss: 0.5189662575721741	F1-Score: 1.0
14.	Loss: 0.5176694989204407	F1-Score: 1.0
15.	Loss: 0.517448902130127	F1-Score: 1.0
16.	Loss: 0.5147578716278076	F1-Score: 1.0
17.	Loss: 0.5147565007209778	F1-Score: 1.0
18.	Loss: 0.5121819376945496	F1-Score: 1.0
19.	Loss: 0.5123520493507385	F1-Score: 1.0
20.	Loss: 0.5116482377052307	F1-Score: 1.0
21.	Loss: 0.5092971920967102	F1-Score: 1.0
22.	Loss: 0.5085961818695068	F1-Score: 1.0
23.	Loss: 0.5067105889320374	F1-Sco

Some weights of the model checkpoint at bert-base-multilingual-uncased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


[+] Data vectorized correctly
1.	Loss: 0.7620685696601868	F1-Score: 0.0
2.	Loss: 0.6323961019515991	F1-Score: 1.0
3.	Loss: 0.527485728263855	F1-Score: 1.0
4.	Loss: 0.500312328338623	F1-Score: 1.0
5.	Loss: 0.484281063079834	F1-Score: 1.0
6.	Loss: 0.491383820772171	F1-Score: 1.0
7.	Loss: 0.4945339560508728	F1-Score: 1.0
